# Log Decoder Development

No need to care about this file, all the content will be merged with raadpy

In [4]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import csv

In [16]:
# Parse a command
def desc_finder(line:str,cmdlist,outputs,i,time,failed_idx):
    """Parse a command and return its status and description

    Args:
        line (str): The string of the command
        cmdlist (pd.DataFrame): pandas data frame with the commands and their equivalent messages
        outputs (_type_): _description_
        i (_type_): _description_
        time (_type_): _description_
        failed_idx (_type_): _description_

    Returns:
        _type_: _description_
    """

    # Get the description
    status = 1
    splt = line.split(' ')

    # define end of log file
    if splt[-1] == 'SE0>':
        # desc = 'LOG END'
        desc = [-1,17]
        text = "LOG END"

    # define commands from the command file
    elif 'txrx' in splt[1]:
        node,port,msg = int(splt[2]),int(splt[3]),str(splt[5])

        index = cmdlist.loc[(cmdlist['NODE']==node) & (cmdlist['PORT']==port) & (cmdlist['Message'].str.startswith(msg)),['ID_COMMAND_Proposed','ID_in_Graph']]
        
        # include the power shutdown
        if node == 4:
            index = cmdlist.loc[(cmdlist['NODE']==node) & (cmdlist['PORT']==port),['ID_COMMAND_Proposed','ID_in_Graph']]

        # include the custom scenario
        if port == 9:
            index = cmdlist.loc[(cmdlist['NODE']==node) & (cmdlist['PORT']==port),['ID_COMMAND_Proposed','ID_in_Graph']]
        

        #If index did not find anything
        if len(index) == 0:
            desc = [] 
            text = "not a payload cmnd"
            
        # if command found in command list
        else:
            desc = list(index.to_numpy()[0])
            text = "cmnd found in list"
        

        if i in [fid for fid in failed_idx]:
            time = time + (float(splt[4])/1000)
            status = -1
        

    elif splt[1] == 'delay':
        time = time + (float(splt[2])/1000)
        desc = []
        text = "delay of"

    elif splt[1] == 'delayuntil':
        time = float(splt[2])
        desc = []
        text = "delay until"

    elif 'read' in splt[1]:
        time = float(outputs[i][0].split(' ')[3])
        desc = []
        text = "read the current time"

    else:
        desc = []
        text = "unidentified command"
    
    return desc,time,status,text

# Decode a logfile
def decode_log(filename:str="../../../Data/Logs/light1-1124v2-se-log.txt"):
    """Take a log file and parse its commands to uncover thier timestamps

    Args:
        filename (str, optional): The filepath and filename of the log file. Defaults to "../../../Data/Logs/light1-SD-1016-se-log.txt".

    Returns:
        _type_: Log, commands, outputs, description, failed_idx, loglines_array
    """

    # Load the logfile
    logfile = open(filename)
    cmdlist = pd.read_csv("command_list.csv")

    # Load the lines
    loglines = logfile.readlines()

    # Close the file
    logfile.close()

    # Create an array with the lines
    commands    = []
    outputs     = []
    idx         = []
    description = []
    times       = []
    fails       = []
    failed_idx  = []
    text_desc   = []

    # Get commands and their indices
    for i, line in enumerate(loglines):
        if 'SE0>' in line:
            commands.append(line)
            idx.append(i)

    # Check if the last command was empty
    if loglines[-1] != 'SE0>':
        commands.append('SE0>')
        idx.append(len(loglines))

    # Get the command output
    for i in range(len(idx)-1):
        out = []
        if 'SE0>#' not in commands[i]:
            for j in range(idx[i]+1,idx[i+1]):
                out.append(loglines[j])
        
        outputs.append(out)

    # Create the dictionary
    log = [[command,output] for command,output in zip(commands,outputs)]

    # Check which commands executed correcly
  

    # Find the commands that failed
    for i in range(len(commands)-1):
        for output in log[i][1]:
            if 'FAIL' in output:
                failed_idx.append(i)


    # Initialize time at 0s
    time = 0
    
    # Give commands and use the finder function to pull the description, time ran, and if the command failed.
    for k, id in enumerate(idx):
        c = commands[k]
        cmd = c.split('\n')[0]
        desc,time,failed,text = desc_finder(cmd,cmdlist,outputs,k,time,failed_idx)

        if len(desc) != 0:
            description.append(desc)
            times.append(time)
            fails.append(failed)
            text_desc.append(text)
    tempname = 'templog.csv'
    fileDir = os.path.join('Defined_Logs')

    if not os.path.exists(fileDir):
        os.makedirs(fileDir)
    
    filePath = os.path.join(fileDir, tempname)

    #Name the headers of the CSV File
    header = ['status','time','description','ID_in_Graph']

    loglines_array = []

    #Clear the file and write from scratch
    with open(filePath, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)

        for i in range(len(description)):
            g = (fails[i], times[i], description[i][0],description[i][1])#,text_desc[i])
            loglines_array.append(g)
            writer.writerow(g)

    # Return everything else
    return log, commands[:-1], outputs, description, failed_idx, loglines_array


In [17]:
log, commands, outputs, description, failed_idx, loglines_array = decode_log()

In [18]:
for i in loglines_array:
    print (i)
# print(loglines_array)

(1, 1657859400.0, 33, 16, 'cmnd found in list')
(1, 1657859400.0, 33, 16, 'cmnd found in list')
(1, 1657859400.0, 33, 16, 'cmnd found in list')
(1, 1657859400.01059, 33, 16, 'cmnd found in list')
(1, 1657859401.01059, 33, 16, 'cmnd found in list')
(1, 1657859402.01059, 33, 16, 'cmnd found in list')
(1, 1657859592.033844, 25, 11, 'cmnd found in list')
(1, 1657859602.033844, 26, 11, 'cmnd found in list')
(1, 1657859612.033844, 25, 11, 'cmnd found in list')
(1, 1657859622.033844, 26, 11, 'cmnd found in list')
(1, 1657859661.980255, 27, 12, 'cmnd found in list')
(1, 1657859671.980255, 28, 12, 'cmnd found in list')
(1, 1657859681.980255, 27, 12, 'cmnd found in list')
(1, 1657859691.980255, 28, 12, 'cmnd found in list')
(1, 1657859751.984924, 9, 8, 'cmnd found in list')
(1, 1657859752.984924, 13, 1, 'cmnd found in list')
(1, 1657859753.984924, 9, 8, 'cmnd found in list')
(1, 1657859754.984924, 13, 1, 'cmnd found in list')
(1, 1657859814.98941, 1, 0, 'cmnd found in list')
(-1, 1657859827.9894